<a href="https://colab.research.google.com/github/maropu/link-prediction-with-anyburl/blob/master/AnyBURL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# （メモ）AnyBURL: ルールベースのグラフのリンク予測手法
 - 元論文: Anytime Bottom-Up Rule Learning for Knowledge Graph Completion, Proceedings of IJCAL, page.3137-3143, 2019.
   - paper: https://www.ijcai.org/proceedings/2019/435
   - source code: https://web.informatik.uni-mannheim.de/AnyBURL/
 - 近年流行中のグラフ埋め込みに対する当該手法の強み
   - **少ない計算時間/計算リソースで，主要なグラフ埋め込み手法と同等程度の（competitiveな）精度を達成可能**
     - 例えば，NNベースの手法がGPUを用いて10時間以上かけて到達する指標値（MRR）をCPUを用いて数分程度で達成
       - 参考: Knowledge Graph Embedding for Link Prediction: A Comparative Analysis, https://arxiv.org/abs/2002.00819
     - Google ColabのCPUインスタンス（2cores CPU/12GiB Mem）上で**約9分の処理時間でMRRが0.327**（後段のColabの実行結果を参照）
       - DGL(Deep Graph Library)のRGCN(Relational Graph Convolutional Networks)実装では[MRRの最良値が~0.2439と報告](https://github.com/dmlc/dgl/tree/f00cd6efbd00b0273f58c393a617415b5d1d410e/examples/pytorch/rgcn)
       - [Papers With CodeのLink Prediction on FB15k-237を確認すると，2023.3現在のMRR最良値は0.391](https://paperswithcode.com/sota/link-prediction-on-fb15k-237)であるため，SOTAな手法と比較すると精度は低い
   - **パラメータの精度への影響が少ないため，パラメータ最適化の必要性が低い**
     - NNベースの手法におけるハイパーパラメータ最適化の様に学習を何度も繰り返し実行する必要性が基本的にはなく，デフォルトの値で妥当な精度を達成できると主張
   - **予測結果の解釈性が高いこと**
     - それぞれの予測結果に対応するルールをスコア（confidence score）とともに出力可能（後段のColabの実行結果を参照）
 - 当該手法の弱み（と考えられるもの）
   - 低次元の空間に埋め込まれたベクトルはリンク予測以外のタスク（例えば分類）にも利用できるが，AnyBURLで生成したルールはリンク予測のタスクのみでの使用を想定
   - SOTAなNNベースの手法(e.g., HGT)は任意のグラフ構造を潜在的に学習可能だと考えられるが，当該手法はルールの形式を3つに限定（AC1，AC2，C）して，生成するルールのパス長(rule body内のatomic sentencesの数)も事前に指定された最大値を持つため，学習が不可能なグラフ構造が存在する（所謂，述語論理におけるLanguage Bias）
 - ルールベースの手法の活用方法（卑見）
   - NNベースの手法は学習が進むまでにかなりの時間を要するため，**妥当な精度まで学習が進んでいるかの判断の基準値**として，短時間でcompetitiveな精度を達成可能なルールベースの手法を活用することは意味がありそう
 - AnyBURLのルールマイニングの特徴（paper内のAlgorithm 1, page 3139）
   - グラフ上のパス（positive examples）をサンプリングして，パスから一般化したルール（Horn rules: rule head <= rule body）を生成，ルールの形式は循環/非循環パスを表現するための3つの雛形（AC1，AC2，C）を定義
     - （言わずもがなだが）タイトルの"Bottom-UP"は，positive examplesから帰納的に一般化することでルールを生成する手法であることに由来
   - 各ルールはpositive/negative examplesの数から計算されるスコア（confidence score）を保持，スコアの計算に必要なexamplesはサンプリングされたものを用いるためスコアは近似値
   - 高い精度を達成するために循環パスから生成されるルール（形式C）が重要であるが，ランダムウォークでは循環パスが選ばれづらい問題があるため，これを解決するために公開されている実装には強化学習を用いたサンプリング手法が採用されているらしい（下記論文を参照，実装は未確認）
     - Reinforced Anytime Bottom Up Rule Learning for
Knowledge Graph Completion, https://arxiv.org/abs/2004.04412
 

# Computing Environment

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.206
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4400.41
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           12Gi       765Mi       8.2Gi       1.0Mi       3.8Gi        11Gi
Swap:            0B          0B          0B


In [ ]:
# AnyBURL running on JVM
!java --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


# AnyBURL compilation

In [ ]:
%%bash
wget https://web.informatik.uni-mannheim.de/AnyBURL/AnyBURL-23-1-sources.zip
unzip AnyBURL-23-1-sources.zip
cd src
javac de/unima/ki/anyburl/*.java -d build
jar cfv AnyBURL.jar -C build .
mv AnyBURL.jar ..

# Test dataset preparation

In [ ]:
!mkdir data

In [ ]:
# If you want to try another data set, you can replace the command line below;
#  - Wikidata5m, ~20M triples, https://deepgraphlearning.github.io/project/wikidata5m
#  - WN18RR, ~90K triples, https://github.com/TimDettmers/ConvE/blob/master/WN18RR.tar.gz
#  - YAGO3-10, ~1M triples, https://github.com/TimDettmers/ConvE/blob/master/YAGO3-10.tar.gz
#  - FB15k-237, ~300K triples, https://github.com/TimDettmers/ConvE/blob/master/FB15k-237.tar.gz
!wget https://github.com/TimDettmers/ConvE/raw/master/FB15k-237.tar.gz
!tar xvzf FB15k-237.tar.gz -C data

In [ ]:
# 'data' folder must have three data files: train.txt, valid.txt, and test.txt
!ls -alFh data

total 23M
drwxr-xr-x 2 root root 4.0K Mar  3 04:39 ./
drwxr-xr-x 1 root root 4.0K Mar  3 04:39 ../
-rw-rw-r-- 1 1000 1000 1.5M May  4  2015 test.txt
-rw-rw-r-- 1 1000 1000  21M May  3  2015 train.txt
-rw-rw-r-- 1 1000 1000 1.3M May  4  2015 valid.txt


In [ ]:
# Show data stats
!find data -type f | xargs wc -l

  272115 data/train.txt
   17535 data/valid.txt
   20466 data/test.txt
  310116 total


In [ ]:
# A data format must be '{head}\t{relation}\t{tail}'
!head -n 3 data/train.txt

/m/027rn	/location/country/form_of_government	/m/06cx9
/m/017dcd	/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor	/m/06v8s0
/m/07s9rl0	/media_common/netflix_genre/titles	/m/0170z3


# Run rule mining (learning phase)

In [ ]:
# List of rule ming configs (config-learn.properties)
#  - SNAPSHOTS_AT: seconds for the intervals of dumping rule snapshots
#  - PATH_TRAINIG: training data path
#  - PATH_OUTPUT: path for an output file having found rules
#  - WORKER_THREADS: number of threads, this number depends on the 'cat /proc/cpuinfo' result
#  - THRESHOLD_CORRECT_PREDICTIONS: threshold for deciding whether to adopt the rules found. If training data is too large, try to make this value higher
#  - THRESHOLD_CONFIDENCE: the same above
#  - SAFE_PREFIX_MODE: AnyBURL assumes each head/relation/tail identifier starts with an alphabetic character and consists of at least 2 letters. If the assumption does not hold, makes this value true

In [ ]:
%%bash
cat <<EOF > config-learn.properties
SNAPSHOTS_AT = 100,200,300,400,500
PATH_TRAINING = /content/data/train.txt
PATH_OUTPUT = /content/data/rules.txt
WORKER_THREADS = 2
THRESHOLD_CORRECT_PREDICTIONS = 2
THRESHOLD_CONFIDENCE = 1.0E-4
SAFE_PREFIX_MODE = false
EOF

In [ ]:
# NOTE: 'NN' number in '-XmsNNg -XmxNNg' depends on the 'free -h' result

In [ ]:
%%time
!java -Xms12g -Xmx12g -cp AnyBURL.jar de.unima.ki.anyburl.Learn config-learn.properties

reading params from file config-learn.properties
* read 273740 triples
* indexed 10000 triples
* indexed 20000 triples
* indexed 40000 triples
* indexed 80000 triples
* indexed 160000 triples
* set up index for 237 relations, 13782 head entities, and 13380 tail entities
* set up list structure for randomized access searches uring rule learning ...  done
* precomputing random starting points for each relation/direction for the beam search ... done.
* creating worker thread #0
* creating worker thread #1
THREAD-0 starts to work with L=3 C=Cyclic 
THREAD-1 starts to work with L=2 C=Cyclic 
  > 99k |  > 99k 009542 002624 |  > 99k
 000569 |  > 99k 009542 003732 |  > 99k
 000569 |  > 99k 009542 007348 | 014448
 000569 | 073550 009542 007348 | 014448
 000569 | 004619 009542 007348 | 014448
 000000 | 004619 009542 012078 | 014448
 000000 | 004619 023408 008122 | 014448
 000000 | 004619 023408 008122 | 021216
 000000 | 004619 009565 006669 | 021216
 000000 | 004619 005396 006669 | 011213
 00000

In [ ]:
!ls -alFh data | grep rules.txt-

-rw-r--r-- 1 root root  18M Mar  3 04:41 rules.txt-100
-rw-r--r-- 1 root root  36M Mar  3 04:42 rules.txt-200
-rw-r--r-- 1 root root  57M Mar  3 04:44 rules.txt-300
-rw-r--r-- 1 root root  80M Mar  3 04:46 rules.txt-400
-rw-r--r-- 1 root root 111M Mar  3 04:47 rules.txt-500


In [ ]:
# The output format: {predicted: int}\t{correctlyPredicted: int}\t{confidence: double}\t{rule}
#  - https://safran.readthedocs.io/en/latest/manual/input_fmt.html
!head -n 3 data/rules.txt-500

5	4	0.8	/award/award_winner/awards_won./award/award_honor/award_winner(X,/m/02qlg7s) <= /award/award_winner/awards_won./award/award_honor/award_winner(X,/m/01m7pwq)
51	3	0.058823529411764705	/ice_hockey/hockey_team/current_roster./sports/sports_team_roster/position(/m/02hqt6,Y) <= /sports/sports_position/players./sports/sports_team_roster/team(Y,A)
11	9	0.8181818181818182	/award/award_nominee/award_nominations./award/award_nomination/award_nominee(X,/m/01r42_g) <= /award/award_winner/awards_won./award/award_honor/award_winner(X,/m/01r42_g)


# Prediction phase

In [ ]:
# List of prediction configs (config-apply.properties)
#  - PATH_RULES: path for the rule file generated in the rule mining phase
#  - PATH_OUTPUT: path for an output file having prediction reuslts

In [ ]:
%%bash
cat <<EOF > config-apply.properties
TOP_K_OUTPUT = 100
PATH_TRAINING = /content/data/train.txt
PATH_VALID    = /content/data/valid.txt
PATH_TEST     = /content/data/test.txt
PATH_RULES    = /content/data/rules.txt-500
PATH_OUTPUT   = /content/data/prediction.out
WORKER_THREADS = 2
SAFE_PREFIX_MODE = false
EOF

In [ ]:
%%time
!java -Xms12g -Xmx12g -cp AnyBURL.jar de.unima.ki.anyburl.Apply config-apply.properties

* reading params from file config-apply.properties
* writing prediction to /content/data/prediction.out
* read 272115 triples
* indexed 10000 triples
* indexed 20000 triples
* indexed 40000 triples
* indexed 80000 triples
* indexed 160000 triples
* set up index for 237 relations, 13781 head entities, and 13379 tail entities
* read 20466 triples
* indexed 10000 triples
* indexed 20000 triples
* set up index for 224 relations, 8171 head entities, and 6376 tail entities
* read 17535 triples
* indexed 10000 triples
* set up index for 223 relations, 7652 head entities, and 5804 tail entities
* reading rules from /content/data/rules.txt-500, read 676941 rules
* applied confidence threshold of 1.0E-4 and reduced from 676941 to 676941 rules
* applying rules
* indexed 100000 rules for prediction
* indexed 200000 rules for prediction
* indexed 300000 rules for prediction
* indexed 400000 rules for prediction
* indexed 500000 rules for prediction
* indexed 600000 rules for prediction
* indexed an

In [ ]:
!head -n 6 data/prediction.out

/m/08966 /travel/travel_destination/climate./travel/travel_destination_monthly_climate/month /m/05lf_
Heads: /m/08966	0.8078970495500809	/m/0l0mk	0.8078970422951789	/m/03czqs	0.8078231292517006	/m/0dprg	0.6351976358733084	/m/0853g	0.6351976358732897	/m/0k9p4	0.6351976355328623	/m/0fq8f	0.6351976352544533	/m/0mbf4	0.635197635135135	/m/019fh	0.6351425166812698	/m/0mpbx	0.6351405931671437	/m/0hpyv	0.635140593167125	/m/0978r	0.6351405928392769	/m/09b83	0.6351405927337916	/m/0l35f	0.6351391124078624	/m/07_kq	0.6351362921599285	/m/03pzf	0.6351362080965085	/m/0jpkg	0.6351360888136092	/m/01gbzb	0.6351357835141874	/m/0171b8	0.6351357835141874	/m/018lc_	0.6351351351351351	/m/0f2wj	0.30000738154613465	/m/01znc_	0.3	/m/0bzjf	0.25	/m/0chghy	0.22222960376835688	/m/0ckhc	0.2222222222222222	/m/0mgp	0.20000738168663054	/m/062qg	0.20000738154613468	/m/01b8jj	0.2	/m/0dhf5	0.16667212426532324	/m/07348	0.16667212426532324	/m/01jc9d	0.16666666666666666	/m/01jng9	0.16666666666666666	/m/015q02	0.1666666666666

In [ ]:
# NOTE: For fast rule application (prediction), you might be able to use an alternative C++ implementation, named SAFRAN, using logical rules generated by AnyBURL.
#  - https://github.com/OpenBioLink/SAFRAN

# Evaluation

In [ ]:
# List of evaluation configs (config-eval.properties)
#  - PATH_PREDICTIONS: path for the prediction file generated in the prediction phase

In [ ]:
%%bash
cat <<EOF > config-eval.properties
PATH_TRAINING = /content/data/train.txt
PATH_VALID    = /content/data/valid.txt
PATH_TEST     = /content/data/test.txt
PATH_PREDICTIONS = /content/data/prediction.out
TOP_K = 100
EOF

In [ ]:
# Output four columns refer to hits@1, hits@3, hits@10, and MRR, respectively
!java -Xms12g -Xmx12g -cp AnyBURL.jar de.unima.ki.anyburl.Eval config-eval.properties

reading params from file config-eval.properties
* read 272115 triples
* indexed 10000 triples
* indexed 20000 triples
* indexed 40000 triples
* indexed 80000 triples
* indexed 160000 triples
* set up index for 237 relations, 13781 head entities, and 13379 tail entities
* read 17535 triples
* indexed 10000 triples
* set up index for 223 relations, 7652 head entities, and 5804 tail entities
* read 20466 triples
* indexed 10000 triples
* indexed 20000 triples
* set up index for 224 relations, 8171 head entities, and 6376 tail entities
* loading result set at /content/data/prediction.out
0.2432   0.3575   0.4967   0.3269


# Explaining Predictions

In [ ]:
# To generate an prediction file with explanation, you can add a 'PATH_EXPLANATION' config in the 'config-apply.properties' file

In [ ]:
%%bash
cat <<EOF > config-apply.properties
TOP_K_OUTPUT = 3
PATH_TRAINING = /content/data/train.txt
PATH_VALID    = /content/data/valid.txt
PATH_TEST     = /content/data/test.txt
PATH_RULES    = /content/data/rules.txt-500
PATH_OUTPUT   = /content/data/prediction.out
PATH_EXPLANATION = /content/data/explanation.out
WORKER_THREADS = 2
SAFE_PREFIX_MODE = false
EOF

In [ ]:
%%time
!java -Xms12g -Xmx12g -cp AnyBURL.jar de.unima.ki.anyburl.Apply config-apply.properties

In [ ]:
!head -n 18 data/explanation.out

/m/08966 /travel/travel_destination/climate./travel/travel_destination_monthly_climate/month /m/05lf_
Heads:
X 0.8078231292517006 [3](3) -> { /m/03czqs } with explanation: 583	475	0.8147512864493996	/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month(X,Y) <= /travel/travel_destination/climate./travel/travel_destination_monthly_climate/month(X,A), /base/localfood/seasonal_month/produce_available./base/localfood/produce_availability/seasonal_months(B,A), /base/localfood/seasonal_month/produce_available./base/localfood/produce_availability/seasonal_months(B,Y)
   X 0.7391304347826086 [2](2) -> { /m/0l0mk } with explanation: 41	34	0.8292682926829268	/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month(X,/m/05lf_) <= /travel/travel_destination/climate./travel/travel_destination_monthly_climate/month(X,/m/06vkl)
      X 0.7254901960784313 [1](1) -> { /m/08966 } with explanation: 46	37	0.8043478260869565	/travel/travel_destination